 9.从网上下载或者自己编程实现LDA，试分析金庸作品《天龙八部》中每十回的话题演变情况。   
 
 《天龙八部》数据在`../data/Demi-Gods and Semi-Devils.txt`，**版权归原作者所有，仅作科研参考，请勿用于商业行为，一切后果自负**。  
 


## 数据预处理   

首先我们将文本按章节分开，每一章看作是一篇文档，每篇文档的内容分句顺序存储下来。 

In [50]:
data = []
chapter = []
for line in open('../data/Demi-Gods and Semi-Devils.txt', encoding='utf-8'):
    if not line.startswith("\u3000\u3000"):
        if line.strip() == "":
            continue
        if len(chapter) > 0:
            data.append(chapter)
        chapter = []
    else:
        chapter.append(line)
else:
    data.append(chapter)
    

然后将每篇文档分词，按词袋存储起来。  

In [108]:
import jieba
stop_words = [line.strip() for line in open('../data/stop_words.txt', encoding='utf-8')] + ["\n", "\t", " "]
data = [ [ item
            for para in chapter
                for item in jieba.cut(para, cut_all=True) if item not in stop_words]
        for chapter in data
]

上LDA模型

In [109]:
from gensim.corpora.dictionary import Dictionary
from gensim.models import LdaModel
# 构建词典
common_dictionary = Dictionary(data)
common_corpus = [common_dictionary.doc2bow(text) for text in data]

# 每十章构建一个LDA模型
models = []
for i in range(int(len(common_corpus) / 10)):
    models.append(LdaModel(common_corpus[i*10:(i + 1)* 10], num_topics=20))

    

In [113]:
# 查看每个模型最重要的五个主题以及每个主题的前五个词  

for i in range(int(len(common_corpus) / 10)):
    print("第 %d-%d 章TOP5主题"%(i*10, (i+1)*10) )
    for _, topic in models[0].print_topics(num_topics=5, num_words=5):
        topic = [common_dictionary[eval(i)] for i in re.findall('"(\d*)"', topic)]
        print(topic)
    

第 0-10 章TOP5主题
['木婉清', '云中', '鳄', '夫人', '二娘', '保定', '姑娘', '南海', '钟', '说道', '婆婆', '司空', '镇南', '中', '好', '段正淳', '都', '红棉', '玫瑰', '大', '不知', '神', '师父', '还', '下', '儿子', '不住', '钟万仇', '帝', '慕容', '公子', '灵', '两', '再', '南王', '想', '十余', '剑', '万里', '宝宝', '女郎', '笑', '大理', '高', '杀', '没', '无量', '不由', '觉', '武功']
['夫人', '木婉清', '云中', '鳄', '南海', '二娘', '中', '钟万仇', '说道', '大', '不住', '不知', '司空', '神', '保定', '段正淳', '都', '两', '剑', '慕容', '帝', '没', '十余', '百泉', '钟', '武功', '红棉', '姑娘', '师父', '好', '灵', '杀', '下', '还', '笑', '宝宝', '再', '一声', '不由', '镇南', '公子', '高', '钟夫人', '袍', '想', '大理', '走', '大师', '青', '瞧']
['夫人', '保定', '木婉清', '中', '云中', '说道', '帝', '红棉', '钟', '大', '段正淳', '都', '司空', '鳄', '南海', '钟万仇', '不住', '剑', '神', '姑娘', '不知', '再', '公子', '师父', '二娘', '大理', '一声', '镇南', '下', '好', '两', '无量', '慕容', '瞧', '灵', '没', '百泉', '十余', '想', '宝宝', '笑', '倒', '小', '钟夫人', '杀', '走', '青', '内力', '后', '大师']
['木婉清', '云中', '夫人', '神', '鳄', '南海', '说道', '二娘', '中', '南王', '段正淳', '帝', '镇南', '不住', '不知', '都', '红棉', '好', '十余', '保定', '武功', '钟', '慕容', '下'